In [1]:
# https://machinelearningmastery.com/a-gentle-introduction-to-normality-tests-in-python/

In [64]:
from components.dataset import Dataset_Builder
from components.preprocessing import preprocessing
from components.stats import check_normality
start_minute=0
stop_minute=5
segment_second=30
sampling_rate = 125
log10 = True
bands = {'delta': [1,3],
    'theta': [4,7],
    'alpha': [8,12],
    'beta': [13,30],
    'gamma': [25,43],
    'slow': [4,13],
    'beta_low': [13,17]
    }

dataset = Dataset_Builder(dataset_path='data')\
            .with_sampling_rate(sampling_rate=sampling_rate)\
            .with_marker(start_minute=start_minute, 
                    stop_minute=stop_minute, 
                    segment_second=segment_second)\
            .build()

data, labels, groups = dataset.load_data_all()
data = preprocessing(data=data,sampling_rate=sampling_rate,bands=bands,log10=log10)

In [65]:
import itertools
ch_names = ['Fp1','Fp2','F3','F4','F7','F8','C3','C4','T3','T4','T5','T6','P3','P4','O1','O2']
band_names = ['delta','theta','alpha','beta','gamma','slow','beta_low','relative']
product = itertools.product(band_names, ch_names)
features = [ feature[1] + '_' + feature[0] for feature in product  ]
features.append('alpha_frontal')
features.append('alpha_temporal')
features.append('alpha_asymetries')
features.append('beta_frontal')
features.append('beta_temporal')
len(features)

133

In [66]:
data[labels == dataset.GROUP_STRESSED, :].shape, data[labels == dataset.GROUP_NON_STRESSED, :].shape

((190, 133), (160, 133))

In [67]:
data[:,[131,130,126,127,115,22,109,128,102,21]].shape

(350, 10)

In [68]:
from scipy.stats import ttest_ind

p_values = []
for i in range(133):
    result = ttest_ind(data[labels == dataset.GROUP_STRESSED, i], data[labels == dataset.GROUP_NON_STRESSED, i], equal_var=False)
    # print(result)
    if(result.pvalue < 0.001):
        print(i, features[i], result)
    p_values.append(result.pvalue)
# np.array(p_values).argsort()
print(sum(np.array(p_values) < 0.001))
print(np.array(p_values).argsort()[:10])

0 Fp1_delta Ttest_indResult(statistic=-4.192830371793877, pvalue=3.6271733338022854e-05)
2 F3_delta Ttest_indResult(statistic=-5.142916154005843, pvalue=5.120754755424126e-07)
3 F4_delta Ttest_indResult(statistic=-3.476050245048737, pvalue=0.000586122551184153)
4 F7_delta Ttest_indResult(statistic=-3.890764975562474, pvalue=0.0001270380016913378)
6 C3_delta Ttest_indResult(statistic=3.8857436216831465, pvalue=0.00012270162770637257)
18 F3_theta Ttest_indResult(statistic=-3.508005576600429, pvalue=0.0005134025655686465)
34 F3_alpha Ttest_indResult(statistic=-4.845686731445187, pvalue=1.9044928334451267e-06)
35 F4_alpha Ttest_indResult(statistic=-3.6203566049304388, pvalue=0.0003386081048113957)
36 F7_alpha Ttest_indResult(statistic=-4.146992140347963, pvalue=4.2448674114210996e-05)
40 T3_alpha Ttest_indResult(statistic=-3.5771515641421634, pvalue=0.0003965589349196775)
42 T5_alpha Ttest_indResult(statistic=-4.404597779883738, pvalue=1.4129184489623668e-05)
43 T6_alpha Ttest_indResult(st

In [69]:
from scipy.stats import mannwhitneyu
p_values = []
for i in range(133):
    result = mannwhitneyu(data[labels == dataset.GROUP_STRESSED, i], data[labels == dataset.GROUP_NON_STRESSED, i])
    if(result.pvalue < 0.001):
        print(i, features[i], result)
    p_values.append(result.pvalue)
np.array(p_values).argsort()
print(sum(np.array(p_values) < 0.001))
print(np.array(p_values).argsort()[:10])

0 Fp1_delta MannwhitneyuResult(statistic=10891.0, pvalue=4.899285989267702e-06)
2 F3_delta MannwhitneyuResult(statistic=11011.0, pvalue=8.921033757349702e-06)
6 C3_delta MannwhitneyuResult(statistic=18625.0, pvalue=0.0002816746186633412)
18 F3_theta MannwhitneyuResult(statistic=10798.0, pvalue=3.0462136587402635e-06)
34 F3_alpha MannwhitneyuResult(statistic=10984.0, pvalue=7.80622363136141e-06)
36 F7_alpha MannwhitneyuResult(statistic=11559.0, pvalue=0.00011308218278971546)
40 T3_alpha MannwhitneyuResult(statistic=12000.0, pvalue=0.0006913454094736478)
42 T5_alpha MannwhitneyuResult(statistic=11957.0, pvalue=0.0005847544876142332)
43 T6_alpha MannwhitneyuResult(statistic=12041.0, pvalue=0.0008095491398651458)
45 P4_alpha MannwhitneyuResult(statistic=11019.0, pvalue=9.279552806064959e-06)
66 F3_gamma MannwhitneyuResult(statistic=11972.0, pvalue=0.0006200662023785448)
82 F3_slow MannwhitneyuResult(statistic=11134.0, pvalue=1.6225385367049565e-05)
98 F3_beta_low MannwhitneyuResult(statist

In [36]:
from scipy.stats import ranksums
p_values = []
for i in range(133):
    result = ranksums(data[labels == dataset.GROUP_STRESSED, i], data[labels == dataset.GROUP_NON_STRESSED, i])
    if(result.pvalue < 0.001):
        print(i, features[i], result)
    p_values.append(result.pvalue)
np.array(p_values).argsort()
sum(np.array(p_values) < 0.001)

0 Fp1_delta RanksumsResult(statistic=-5.5619903737049, pvalue=2.6671500830368357e-08)
2 F3_delta RanksumsResult(statistic=-6.047876607781252, pvalue=1.4676729097959636e-09)
3 F4_delta RanksumsResult(statistic=-3.871706601879443, pvalue=0.00010807599594131161)
4 F7_delta RanksumsResult(statistic=-4.064185086884207, pvalue=4.820056711995592e-05)
6 C3_delta RanksumsResult(statistic=5.281714685013753, pvalue=1.2798039761190197e-07)
15 O2_delta RanksumsResult(statistic=-3.616194227048558, pvalue=0.00029896609033146313)
17 Fp2_theta RanksumsResult(statistic=3.8975955209151714, pvalue=9.715250626356679e-05)
18 F3_theta RanksumsResult(statistic=-6.179572413310827, pvalue=6.427544729616915e-10)
22 C3_theta RanksumsResult(statistic=3.98576792632671, pvalue=6.726223020040428e-05)
32 Fp1_alpha RanksumsResult(statistic=-4.05555544720563, pvalue=5.001530570414858e-05)
33 Fp2_alpha RanksumsResult(statistic=-3.297647962391552, pvalue=0.0009749828996169052)
34 F3_alpha RanksumsResult(statistic=-6.48873

43

In [2]:
from scipy.stats import shapiro
alpha = 0.05
stats, ps = [],[]
for i in range(data.shape[1]):
    column = data[:,i]
    stat, p = shapiro(column)
    stats.append(stat)
    ps.append(p)
    if p > alpha:
        print(i, ': Sample looks Gaussian (fail to reject H0)')
    # else:
    #     print('Sample does not look Gaussian (reject H0)')
ps = np.array(ps)
np.where(ps > alpha)

(array([], dtype=int64),)